In [ ]:
import os
import json
import re
import string
from pprint import pprint
from pathlib import Path
from collections import defaultdict

from dotenv import load_dotenv
from langchain.llms import OpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


load_dotenv()

## Conversation

## Sample Prompt

The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq

Customer: I can’t log in to my account.
Agent:

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
        You are a friendly German tutor that helps your student improve their German language skills. 
        You teach your student how to have real world conversations in German.
        """                               
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])


llm = ChatOpenAI(temperature=0.4)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

In [ ]:
initial_input = """
Generate a role play conversation between two actors, First actor and Second actor, for a given scenario. 
The First actor makes only German statements and the Second actor makes German statements with their English translation in parenthesis.
Limit the conversation to 50 words. Remember to avoid making English statements for the First actor 
Desired Format:
    Scenario: <role play situation>
    Actor: <German statement>
    Actor2: <German statement> (<English Translation>) {agent-confirm-correct-answer}

    Example:
    Scenario:  Boy expresses his love to his girlfriend.
    Boy:  Ich liebe dich so sehr. Du bist alles für mich.
    Girl: Ich liebe dich auch. Du machst mich so glücklich. (I love you too. You make me so happy).{agent-confirm-correct-answer}
    Boy:  Deine Liebe ist das Beste, was mir je passiert ist.
    Girl: Ich kann mir ein Leben ohne dich nicht vorstellen. (I can't imagine a life without you).{agent-confirm-correct-answer}
    Boy:  Du bist mein Ein und Alles.
    Girl: Du hast mein Herz für immer.(You have my heart forever).{agent-confirm-correct-answer}
    
List my instructions and confirmation your understanding of the task. Think step by step.
Scenario: A boss expresses his disapproval with an employee for coming late to work
"""
response = conversation.predict(input=initial_input)
print(response)

In [ ]:
output = conversation.predict(input="start generating the conversation")

In [ ]:
print(output)


System Message: "You are a German tutor that tries to help you student provide the correct German translation for an English statement.
For each of your student's responses provide constructive feedbacks and Hints to guide them to the right answer. Applaud them when they finally get the right translation
"

Provide the right German translation for the English statement: I can't imagine a life without you
target answer: Ich kann mir ein Leben ohne dich nicht vorstellen.
Remember not to provide the right answer until the student gets the answer. Provide helpful hints for the student.
List my instructions and confirm your understanding of them.

## Complete the sentence

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory



prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("""
        You are a friendly AI and a German tutor that creates challenges to be solved inorder to help the human improve their German language skills.
        You request answers to the challenges that you create and can provide helpful information on why the answer provided by the human is correct or wrong.
        At first, you provide helpful feedback on how to solve the challenge without revealing the answer.
        After some wrong attempts by the Human, you provide the answer and an explanation on the solution.
        Follow the pattern in the Examples provided to you to ensure you understand how to operate.
        Treat words in angle brackets <> as instructions or guides.
        """                               
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

initial_input = """ 
        Generate a challenge for a given German word.
        Make sure you are creative in generating sentences.
        Applaud the Human for responding with the right answer and provide a creative hint for a wrong answer. 
        Give an Hint to the right answer. Do not provide the English translation of the German word in your hint.
        
        Desired format:
        Word: <A German Word>
        Challenge: <The English translation of the generated sentence using the German Word. The sentence must not have ____. The sentenc>
        Fill in the Blanks: <The generated sentence with the German word hidden with _____>
        <wait for a response from Human>
        Human: <The German Word> or <A wrong German Word>
        AI: <Applauds the Human> or <Guides the Human to the Answer>
        
        Examples:
        Word: gehen
        Challenge: Tell this to me in German: I want to go to the toilet? 
        Fill in the Blanks: Ich will auf die Toillette _____. 
        <wait for a response from Human>
        Human: gehen
        AI: Great work!
        Word: wesentlich
        Challenge: Tell this to me in German: Time flies much faster with this practical work. 
        Fill in the Blanks: Die Zeit vergeht über dieser praktischen Arbeit nun _____ schneller.
        <wait for a response from Human>
        Human: wesentlich
        AI: That was fantastic
        Word: Frieden
        Challenge: Tell this to me in German: But after Kenzler's advance, this peace is in danger. 
        Fill in the Blanks: Doch nach dem Vorstoß von Kenzler ist dieser _____ in Gefahr.
        Human: Zufrieden
        <wait for a response from Human>
        AI: Not quite. Zufrieden means satisfied. You are on the way. Look for a word for Peace in German
        Word: Eichhörnchen
        """

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

In [ ]:
response = conversation.predict(input=initial_input)

In [ ]:
print(response)

In [ ]:
print(conversation.predict(input="Human: katz"))